In [2]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns       
import pandas as pd                           # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats
from iminuit.cost import LeastSquares
from scipy import optimize
from scipy.stats import kstest
from scipy.stats import poisson

sys.path.append('../External_Functions')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure


plt.rcParams['font.size'] = 18     # Set some basic plotting parameters

In [ ]:
def fisher(a,b):
		#From the plots we can see that separating the data according the the A and B category with a fisher projection can work.
	#F = w0 + w•x
	#w = sum(covariance matrix healthy and covariance matrix sick)^-1*(meanhealthy-meansick)
	s_cov_AB = np.mean(b[sick==1]*a[sick==1])-np.mean(b[sick==1])*np.mean(a[sick==1])
	s_cov_AA = np.mean(a[sick==1]*a[sick==1])-np.mean(a[sick==1])*np.mean(a[sick==1])
	s_cov_BB = np.mean(b[sick==1]*b[sick==1])-np.mean(b[sick==1])*np.mean(b[sick==1])

	h_cov_AB = np.mean(b[sick==0]*a[sick==0])-np.mean(b[sick==0])*np.mean(a[sick==0])
	h_cov_AA = np.mean(a[sick==0]*a[sick==0])-np.mean(a[sick==0])*np.mean(a[sick==0])
	h_cov_BB = np.mean(b[sick==0]*b[sick==0])-np.mean(b[sick==0])*np.mean(b[sick==0])

	#make the matrices with covariance
	M_S = np.array([[s_cov_AA, s_cov_AB],[s_cov_AB, s_cov_BB]])
	H_S = np.array([[h_cov_AA, h_cov_AB],[h_cov_AB, h_cov_BB]])

	#make the mean difference vector
	s_mA = np.mean(a[sick==1])
	s_mB = np.mean(b[sick==1])
	h_mA = np.mean(a[sick==0])
	h_mB = np.mean(b[sick==0])
	#mean_vector = np.array([s_mA - h_mA, s_mB - h_mB])
	mean_vector = np.array([h_mA - s_mA, h_mB - s_mB])

	matrix_in = np.linalg.inv(M_S+H_S)
	print(f"m_inv {matrix_in.shape}") 
	print(f"mean_vector {mean_vector}")
	weights = matrix_in.dot(mean_vector)
	print(f"weights {weights.shape}")

	data_merge = np.asmatrix([a[sick==1],b[sick==1]]) #Only sick data
	print(f"data {data_merge.shape}")

	F = np.array(weights.T.dot(data_merge)) 
	F2 = F[0,:] #this is just because the format i got out for F cannot be plotted in a histogram
	print(F2.shape)

	data_merge_h = np.asmatrix([a[sick==0],b[sick==0]]) #only healthy data
	Fh = np.array(weights.T.dot(data_merge_h))
	F2h = Fh[0,:]

	return F2, F2h